In [4]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from nltk.tokenize import word_tokenize #import library nltk - tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import library sastrawi untuk
from textblob import TextBlob
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
import np

from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import TweetTokenizer
from cleantext import clean
from nltk.stem import PorterStemmer


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [5]:
# Emoji Senang
emoticons_happy= set([
    ':-)',':)',';)',':o)',':]',':3',':c)',':>','=]','8)','=}',
    ':^)',':-D',':D','8-D','8D','x-D','XD','X-D','xD','wkwkwkw','haha'
])

# Emoji Sedih
emoticons_sad =set([
    ':l',':-/','>:/',':S','>:]',':@',':-(',':[',':-||','-l',':<',
    ':-[',':-<','=\\','=/','>:(',':(','>_<','hhh'
])

# all emoticons (hapy+sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [6]:
SLANKWORDS = {
    "skrg":"sekarang",
    "ni":"ini","tp":"tetapi","ms":"masih",
    "iso":"bisa",
    "syaanggg":"sayang",
    "sebelom":"sebelum",
    "tbtb":"lalu",
    "muali":"mulai",
    "tak":"tidak",
    "yt":"youtube","nimbrung":"mengikuti","ngasal":"asal",
    "sampe":"sampai","hp":"handphone","aja":"saja","nihhh":"nih","pastiii":"pasti","kagaa":"tidak","blm":"belum",
    "anjim":"anjing",
    "yg":"yang",
    "diyt":"di youtube",
    "gk":"tidak",
    "aing":"saya",
    "hny":"hanya",
    "dr":"dari",
    "gak":"tidak",
    "plg":"paling",
    "sdh":"sudah",
    "adlh":"adalah",
    "vs":"versus",
    "ndak":"tidak",
    "ntr":"nanti",
    "klo":"kalau",
    "dlu":"dulu",
    "krn":"karena","kadrun":"kadal gurun",
    "gw":"saya","sssaja":"saja",
    "gua":"saya",
    "gmn":"bagaimana",
    "gw":"saya",
    "tdk":"tidak","kaga":"tidak",
    "kpd":"kepada",
    "unk":"untuk",
    "t4gar":"tagar",
    "lg":"lagi",
    "rs":"rumah sakit",
    "trus":"terus","go out:(":"pergi sad",
    "w":"saya",
    "udah":"sudah","susaaah":"susah","bgt":"sangat",
    "dsr":"dasar",
    "srh":"suruh",
    "dg":"dengan","skuyy":"","lgsg":"",
    "jg":"juga","trs":"terus",
    "covid-19":"korona",
    "sars-cov-2":"korona",
    "tgr":"tagar",
    "aq":"saya","ssaja":"saja",
    "gr2":"karena","dgn":"dengan",
    "kovid-19":"korona",
    "gue":"saya","alloh":"Allah",
    "jd":"jadi","plis jng":"mohon jangan",
    "gpp":"tidak apa","onlen":"online",
    "ato":"atau",
    "ga":"tidak","hhhhhh":"sad",
    "engtidak":"tidak","xe x xa":"",
    "covid 19":"korona","kiyowoo":"imut",
    "kelar":"selesai",
    "kek":"seperti",
    "pankapanlah":"nanti","uu":"undang-undang",
    "pesepedamarak":"pesepeda marak",
    "cepatt":"cepat","drpd":"daripada",
    "ntar":"nanti","sampek":"sampai","udh":"sudah","grgr":"karena","ngtidak":"tidak","ak":"saya","tpi":"tetapi","knp":"kenapa"
}

In [286]:
class Sentys:
    TWEET = 'tweet'
    DATA_SLANK = None
    SENTIMENTAL_POLARITY = 'POLARITY'
    SENTIMENTAL_SUBJECTIVE = 'SUBJECTIVE'
    SENTIMENTAL_SENTIMENT = 'SENTIMENT'
    SENTIMENTAL_THRESHOLD = 0
    
    def __init__(self, excel, fields, slankwords):
        self.__excel = excel
        self.__fields = fields
        self.__data = None
        self.__data_frame = None
        self.__full_text = None
        self.__total_polarity = 0
        Sentys.DATA_SLANK = slankwords
        
    def execute(self):
        self.__nltk_dependency()
        self.__build_data()
        self.__build_data_frame()
        self.__build_remove_slank()
        self.__build_remove_patterns()
        self.__build_clean_tweet()
        self.__build_sentiment()
        
    def __build_data(self):
        self.__data = pd.read_excel(self.__excel,
                                  header=None,
                                  names=self.__fields)
        
    def __build_data_frame(self):
        self.__data_frame = pd.DataFrame(self.__data)
        
    def __build_remove_slank(self):
        self.__full_text = self.__data_frame[Sentys.TWEET].values.tolist() #masukan data kedalam list
        
        for index, text in enumerate(self.__full_text):
            text = text.lower()
            splitlines = word_tokenize(text)
            for word in splitlines:
                if word in Sentys.DATA_SLANK:
                    text = text.replace(word, str(Sentys.DATA_SLANK[word]));
                    self.__full_text[index] = text
        
        self.__data_frame['slank_text'] = self.__full_text
    
    @staticmethod
    def __vectorize_remove_patterns(tweet, pattern):
        r = re.findall(pattern, tweet)
        
        for i in r:
            tweet=re.sub(i,'', tweet)
            
        return tweet
        
            
    def __build_remove_patterns(self):
        self.__data_frame['remove_user'] = np.vectorize(
            Sentys.__vectorize_remove_patterns
        )(self.__data_frame['slank_text'] ,"@[\w]*")
    
    @staticmethod
    def __clean_tweet(tweet):

        stopwords_indonesia = stopwords.words('indonesian')
        factory=StemmerFactory()
        stemmer =factory.create_stemmer()
#         print(tweet)
         # Membuang angka
#         tweet = re.sub('[0-9]+','',tweet)

#         # Membuang stock market stiker seperti GE
#         tweet = re.sub(r'\$\w*b','',tweet)

#         # membuang style retweet RT text "RT"
#         tweet = re.sub(r'^RT[\s]+','',tweet)

#         # membuang hashtag cuman tanda # nya doang
#         tweet = re.sub(r'#','',tweet)
        
#         tweet = re.sub(r'\$\w*','',tweet)
#         tweet = re.sub(r'^RT[\s]+','',tweet)
#         tweet = re.sub(r'^rt[\s]+','',tweet)
#         tweet = re.sub(r'https?:\/\/.*[\r\n]*','',tweet)
#         tweet = re.sub(r'#','',tweet)
#         tweet = re.sub(r',',' ',tweet)
#         tweet = re.sub('[0-9]+','',tweet)

#         tokenizer = TweetTokenizer(preserve_case=False,strip_handles=True,reduce_len=True)
#         tweet_tokens = tokenizer.tokenize(tweet)

        tweets_clean = []
        data = clean(tweet.strip(),
            fix_unicode=False,               # fix various unicode errors
            to_ascii=True,                  # transliterate to closest ASCII representation
            lower=True,                     # lowercase text
            no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
            no_urls=True,                  # replace all URLs with a special token
            no_emails=True,                # replace all email addresses with a special token
            no_phone_numbers=True,         # replace all phone numbers with a special token
            no_numbers=True,               # replace all numbers with a special token
            no_digits=True,                # replace all digits with a special token
            no_currency_symbols=True,      # replace all currency symbols with a special token
            no_punct=True,                 # remove punctuations
            no_emoji=True,
            replace_with_punct="",
            replace_with_digit="",
            replace_with_number="",
            replace_with_url='',
#             normalize_whitespace=False,
        )
        
#         for word in tweet_tokens:
#             if(word not in stopwords_indonesia and #remove stropwords
#                   word not in emoticons and #removes emoticon
#                       word not in string.punctuation): #remove punctuation
#                  # tweets_clean.append(word)
#                 stem_word=stemmer.stem(word) #remove word
#                 tweets_clean.append(stem_word)
        
        return str(data.rstrip('\r\n'))
    
    def __sentimental(self, text, key):
        analysis = TextBlob(text)
        self.__total_polarity += analysis.polarity
        
        if key == Sentys.SENTIMENTAL_SUBJECTIVE:
            return analysis.sentiment[1]
        elif key == Sentys.SENTIMENTAL_POLARITY:
            return analysis.polarity
        elif key == Sentys.SENTIMENTAL_SENTIMENT:
            if analysis.polarity == Sentys.SENTIMENTAL_THRESHOLD:
                return 'Neutral'
            elif analysis.polarity > Sentys.SENTIMENTAL_THRESHOLD:
                return 'Positive'
            else:
                return 'Negative'
    
    def __build_clean_tweet(self):
        self.__data_frame['tweet'] = self.__data_frame['remove_user'].apply(
            lambda x: Sentys.__clean_tweet(x)
        )
        
        self.__data_frame.drop_duplicates(subset="tweet",keep='first',inplace=True)
        self.__data_frame.sort_values("tweet",inplace=True)
        
    def __build_sentiment(self):
        self.__data_frame['polarity'] = self.__data_frame['tweet'].apply(
            lambda x: self.__sentimental(x, Sentys.SENTIMENTAL_POLARITY)
        )
        self.__data_frame['subjective'] = self.__data_frame['tweet'].apply(
            lambda x: self.__sentimental(x, Sentys.SENTIMENTAL_SUBJECTIVE)
        )
        self.__data_frame['sentiment'] = self.__data_frame['tweet'].apply(
            lambda x: self.__sentimental(x, Sentys.SENTIMENTAL_SENTIMENT)
        )
        
    def get_data_frame(self):
        return self.__data_frame
    
    def get_sentiment_result(self):
        if self.__total_polarity/100 > 0:
            return "Positive", self.__total_polarity/100
        
        elif self.__total_polarity/100 == 0:
            return "Netral", self.__total_polarity/100
        
        elif self.__total_polarity/100 < 0:
            return "Negative", self.__total_polarity/100
        
    def save(self, name):
        self.__data_frame.to_csv(f'{name}.csv', encoding='utf8', index=True)
        
    def save_db(self, orm):
        pass
        
    def __nltk_dependency(self):
        nltk.download('punkt')
        nltk.download('stopwords')

In [287]:
sp = Sentys('TESTING.xlsx', ['posting_id','user_name','tweet'], SLANKWORDS)
sp.execute()
sp.get_data_frame()
sp.get_sentiment_result()
sp.save('result')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\binnafis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\binnafis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [290]:
def knn(filename, fields):
    data = pd.read_csv(filename, names=fields)
    X = data.tweet
    y = data.sentiment

    #Using CountVectorizer to convert text into tokens/features
    vect = CountVectorizer(stop_words='english', ngram_range = (1,1), max_df = 0.95, min_df = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3)

    #Using training data to transform text into counts of features for each message
    vect.fit(X_train)
    X_train_dtm = vect.transform(X_train)
    X_test_dtm = vect.transform(X_test)

    #Accuracy using KNN Model
    KNN = KNeighborsClassifier(n_neighbors =3)
    KNN.fit(X_train_dtm, y_train)
    y_pred = KNN.predict(X_test_dtm)
    print('\nK Nearest Neighbors (NN = 3)')
    print('Accuracy Score: ',metrics.accuracy_score(y_test,y_pred),sep='')
    print('Confusion Matrix: ',metrics.confusion_matrix(y_test,y_pred), sep = '\n')
    
knn('result.csv', ['tweet', 'sentiment'])


K Nearest Neighbors (NN = 3)
Accuracy Score: 0.9523809523809523
Confusion Matrix: 
[[ 0  1  0]
 [ 0 60  0]
 [ 0  2  0]]


In [275]:
# data = pd.read_csv('data_korona_sentimen.csv',names=['Tweet','sentimen'])
# X = data.Tweet
# y = data.sentimen

# #Using CountVectorizer to convert text into tokens/features
# vect = CountVectorizer(stop_words='english', ngram_range = (1,1), max_df = 0.95, min_df = 1)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3)

# #Using training data to transform text into counts of features for each message
# vect.fit(X_train)
# X_train_dtm = vect.transform(X_train)
# X_test_dtm = vect.transform(X_test)

class SentysVerify:
    """
    MIT License
    created by Junox (yanwarsolah@gmail.com)
    This program for free use to education program.
    """
    TEST_SIZE = 0.3
    K = 3
    MAX_DF = 0.95
    MIN_DF = 1
    NGRAM = (1, 1)
    LANG = 'english'
    
    def __init__(self, excel, fields):
        self.__excel = excel
        self.__fields = fields
        self.__x = None
        self.__y = None
        self.__x_train = None
        self.__x_test = None
        self.__y_train = None
        self.__y_test = None
        self.__vector = None
        self.__x_train_dtm = None
        self.__x_test_dtm = None
        self.__y_train_dtm = None
        self.__y_test_dtm = None
        self.__y_pred = None
        self.__classifier = None
        
    def __build_vector(self):
        self.__vector = CountVectorizer(
            stop_words=SentysVerify.LANG, 
            ngram_range=SentysVerify.NGRAM,
            max_df=SentysVerify.MAX_DF,
            min_df=SentysVerify.MIN_DF
        )
        
    def __build_vector_fit(self):
        self.__vector.fit(self.__x_train)
        self.__x_train_dtm = self.__vector.transform(self.__x_train)
        self.__x_test_dtm = self.__vector.transform(self.__x_test)

    def __build_train_and_test(self):
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = train_test_split(
            self.__x, 
            self.__y, 
            test_size=SentysVerify.TEST_SIZE
        )
        
    def execute(self):
        self.__build_data()
        self.__build_vector()
        self.__build_data_frame()
        
        self.__build_train_and_test()
        self.__build_vector_fit()
        self.__build_accuracy()
        
    def __build_data(self):
        self.__data = pd.read_excel(
            self.__excel, 
            header=None, 
            names=self.__fields
        )
        self.__x = self.__data.tweet
        self.__y = self.__data.sentiment
        
    def __build_accuracy(self):
        self.__classifier = KNeighborsClassifier(n_neighbors=SentysVerify.K)
        self.__classifier.fit(self.__x_train_dtm, self.__y_train)
        self.__y_pred = KNN.predict(self.__x_test_dtm)
        
    def get_accuracy(self):
        return metrics.accuracy_score(self.__y_test, self.__y_pred)

    def get_confusion_matrix(self):
        return metrics.confusion_matrix(self.__y_test, self.__y_pred)
        
    def __build_data_frame(self):
        self.__data_frame = pd.DataFrame(self.__data)
#         print(self.__data_frame)

In [277]:
sv = SentysVerify('result.xlsx', ['tweet','sentiment'])
sv.execute()

AttributeError: 'int' object has no attribute 'lower'